#### 导入库

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "3"


#### 文档加载&切分&检索

In [2]:
! python ingest.py # 命令行 python3 ./DB/Process/ingest.py 制作向量数据库

python: can't open file '/home/linjw/AGI/llama2/langchain/ingest.py': [Errno 2] No such file or directory


In [3]:
### 文档查询 去向量数据库匹配相关代码
doc_query = "人工智能 突破技术 未来技术 塑造未来 革命 新一代技术 未来产业"


### 递归总结, 根据查询出来的向量数据库相关片段 循环输入进llm进行要素提取
refine_query = """
根据获取的信息，总结并提炼出人工智能的突破性技术：
1. 技术1: [描述]
2. 技术2: [描述]
3. 技术3: [描述]
4. 技术4: [描述]
5. 技术5: [描述]
6. 技术6: [描述]
7. 技术7: [描述]
8. 技术8: [描述]
9. 技术9: [描述]
10. 技术10: [描述]
"""


# 最终总结输出 总结递归的最后输出
summary_query = """在已知的信息中，确定并列出人工智能的突破性技术"""


# 回答模板 将summary_query的输出格式化为对象
response_schemas = [
    ResponseSchema(name="技术1", description="技术1的简要描述"),
    ResponseSchema(name="技术2", description="技术2的简要描述"),
    ResponseSchema(name="技术3", description="技术3的简要描述"),
    ResponseSchema(name="技术4", description="技术4的简要描述"),
    ResponseSchema(name="技术5", description="技术5的简要描述"),
    ResponseSchema(name="技术6", description="技术1的简要描述"),
    ResponseSchema(name="技术7", description="技术2的简要描述"),
    ResponseSchema(name="技术8", description="技术3的简要描述"),
    ResponseSchema(name="技术9", description="技术4的简要描述"),
    ResponseSchema(name="技术10", description="技术5的简要描述"),
]


In [4]:
sys.path.append(os.path.dirname(os.path.realpath('__file__'))+"/../")
import config.conf as cfg
from chromadb.config import Settings

# Define the folder for storing database
PERSIST_DIRECTORY = cfg.PERSIST_DIRECTORY.replace("/langchain","")
# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embeddings_model_name = cfg.EMBEDDINGS_MODEL_NAME.replace("/langchain","")
persist_directory = cfg.PERSIST_DIRECTORY.replace("/langchain","")
model_path = cfg.MODEL_PATH.replace("/langchain","")
model_n_ctx = cfg.MODEL_N_CTX
model_n_batch = cfg.MODEL_N_BATCH
target_source_chunks = 4
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)

In [5]:
cfg.EMBEDDINGS_MODEL_NAME

'/home/linjw/AGI/llama2/langchain/checkpoints/paraphrase-multilingual-MiniLM-L12-v2'

##### 普通检索

In [16]:
retriever = db.as_retriever(search_type="mmr",search_kwargs={"k": 20})

# retriever = db.as_retriever(    
#     search_type="mmr",
#     search_kwargs={'k': 20, 'lambda_mult': 0.25})


In [19]:
docs = retriever.get_relevant_documents(doc_query)
[doc.page_content for doc in docs]

['their overall digital transformation maturity journey .\n人工智能未来\n已来，是机遇\n还是挑战\n人工智能产业近年来发展迅猛，渗透了各行各业，而新技术的革新往往\n是一把“双刃剑”，在提升企业商业利益的同时也会带来一定的风险。\n本报告对各行业领域中超过100家企业进行了调研，并根据整合的数据\n进行了深度洞察分析，其中包括人工智能未来5年的用途和影响，以及\n人工智能为企业带来的商业利益和风险。\n大中华区人工智能成熟度调研\n35',
 '01  <<\n新一代人工智能产业白皮书（2019 年）\n——主要应用场景研判\n新一代人工智能作为全球新一轮科技和产业变革的关键驱动力，将进一步释放科技\n革命和产业变革积蓄的巨大能量，持续探索新一代人工智能应用场景，将重构生产、分配、\n交换、消费等经济活动各环节，催生新技术、新产品、新产业、新业态、新模式，形成\n从宏观到微观各领域的智能化新需求，并创造新的经济发展的强大引擎，引发经济结构\n重大变革，深刻改变人类生产生活方式和思维模式，实现社会生产力的整体跃升。\n全球新一代人工智能产业依赖强大的技术创新积累优势，以谷歌、英特尔、微软、\n亚马逊等跨国大型科技企业为主导，充分发挥其强大的资源整合能力与持续创新功能，\n不断加快基础层底层技术研发与应用产品实践步伐，围绕智能硬件与软件核心算法产业\n上下游进行有效部署。随着技术的不断成熟和底层技术框架的开源，吸引创新企业不断\n涌入，推动产业规模持续加速增长。2018 年全球新一代人工智能产业规模超过 555.7 亿\n美元，预计 2019 年产业规模将突破 718 亿美元，带动 2022 年产业规模将超过 1630.2',
 '27\n人工智能未来发展洞察\n全球趋势一：人工智能企业将以技术共享的方式\n推动技术迭代\n人工智能处于高速发展阶段，技术快速迭代，企\n业研发新技术需要耗费大量的人力及时间成本，\n变相减缓人工智能行业发展速度。因此，人工智\n能企业将会结成发展联盟，共享彼此的专利技术，\n共同推进AI通用技术发展，缩短人工智能行业进\n入成熟期的时间。\n全球趋势二：人工智能企业持续挖掘多元场景应\n用，以期稳定市场期望值\n根据Gartner人工智能技术成熟度曲线显示，有大\n量人工智能应

In [8]:
for doc in docs:
    print(doc.metadata)


{'author': '', 'creationDate': "D:20201022153906+08'00'", 'creator': 'Adobe InDesign 15.0 (Windows)', 'file_path': '/home/linjw/AGI/llama2/DB/Process/source/2020年  大中华区人工智能成熟度调研——中国大陆，解码2020，展望数字未来-安永x微软.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20201112143019+08'00'", 'page': 34, 'producer': '99pdf.com', 'source': '/home/linjw/AGI/llama2/DB/Process/source/2020年  大中华区人工智能成熟度调研——中国大陆，解码2020，展望数字未来-安永x微软.pdf', 'subject': '', 'title': '', 'total_pages': 81, 'trapped': ''}
{'author': '', 'creationDate': '', 'creator': '', 'file_path': '/home/linjw/AGI/llama2/DB/Process/source/2019年  新一代人工智能产业白皮书：主要应用场景研判-200114.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200515121112+08'00'", 'page': 8, 'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': '/home/linjw/AGI/llama2/DB/Process/source/2019年  新一代人工智能产业白皮书：主要应用场景研判-200114.pdf', 'subject': '', 'title': '中国电子学会-2019新一代人工智能产业白皮书：

##### LLM压缩检索

In [9]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain.llms import OpenAI
# from langchain.retrievers.self_query.base import SelfQueryRetriever
# from langchain.chains.query_constructor.base import AttributeInfo
 
# #定义元数据的过滤条件
# metadata_field_info = [
#     AttributeInfo(
#         name="source",
#         description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
#         type="string",
#     ),
#     AttributeInfo(
#         name="page",
#         description="The page from the lecture",
#         type="integer",
#     ),
# ]
 
# #创建SelfQueryRetriever
# document_content_description = "Lecture notes"
# llm = OpenAI(temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     document_content_description,
#     metadata_field_info,
#     verbose=True
# )
# def pretty_print_docs(docs):
#     print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    
# # Wrap our vectorstore
# llm = OpenAI(temperature=0)
# compressor = LLMChainExtractor.from_llm(llm)
 
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=db.as_retriever()
# )

# question = "what did they say about matlab?"
# compressed_docs = compression_retriever.get_relevant_documents(question)
# pretty_print_docs(compressed_docs)

#### 加载模型

In [10]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
model = HuggingFacePipeline.from_model_id(model_id=model_path,
        task="text-generation",
        device=0,
        pipeline_kwargs={
            "max_new_tokens": 4096, 
            "do_sample": True,
            "temperature": 0.2,
            "top_k": 1,
            "top_p": 0.9,
            "repetition_penalty": 1.1},
        model_kwargs={
            "torch_dtype": "auto",
            "low_cpu_mem_usage": True}
        )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


##### 定义目标模板

In [11]:
qa_chain_refine = RetrievalQA.from_chain_type(
    model,
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type="refine"
)
 
result = qa_chain_refine({"query": refine_query})
result["result"]


'\n\nRevised answer:\n1. 技术1: 深度学习和神经网络的发展使得机器能够模拟人类大脑的工作方式，从而实现更准确和复杂的任务。\n2. 技术2: 自然语言处理（NLP）的技术进步使计算机能够理解和生成人类语言，推动了智能助手、语音识别等领域的发展。\n3. 技术3: 强化学习在机器人控制、游戏策略等方面取得了重大突破，通过试错和奖励机制来优化决策过程。\n4. 技术4: 图像识别和计算机视觉领域的进展，包括卷积神经网络（CNN）的应用，使得机器能够自动识别和分类图像内容。\n5. 技术5: 自动驾驶技术的发展，利用传感器数据和机器学习算法进行实时路况分析和车辆控制，实现了无人驾驶的目标。\n6. 技术6: 人机交互界面的设计和改进，如虚拟现实（VR）和增强现实（AR）技术，为用户提供了更加沉浸式的体验。\n7. 技术7: 区块链技术在人工智能中的应用，可以提供去中心化的数据存储和交换平台，提高了数据安全性和隐私保护能力。\n8. 技术8: 量子计算的发展，有望大幅提升计算能力和解决复杂问题的能力，对人工智能领域具有潜在影响。\n9. 技术9: 联邦学习的方法，可以在不共享个人敏感信息的情况下进行模型训练和推理，提高了数据隐私保护的水平。\n10. 技术10: 迁移学习的方法，可以通过将已经'

### 整理回答

#### 数据解析器

In [12]:
que =result["result"]

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "以下为背景知识：\n"
    "{context_str}"
    "\n"
    "{format_instructions}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=initial_qa_template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions,"context_str":que}
)
_input = prompt.format_prompt(question=summary_query)
output = model(_input.to_string())
print(output)
# output_parser.parse(output)
 

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 以下是人工智能的突破性技术列表：

1. 技术1: 深度学习和神经网络的发展使得机器能够模拟人类大脑的工作方式，从而实现更准确和复杂的任务。
2. 技术2: 自然语言处理（NLP）的技术进步使计算机能够理解和生成人类语言，推动了智能助手、语音识别等领域的发展。
3. 技术3: 强化学习在机器人控制、游戏策略等方面取得了重大突破，通过试错和奖励机制来优化决策过程。
4. 技术4: 图像识别和计算机视觉领域的进展，包括卷积神经网络（CNN）的应用，使得机器能够自动识别和分类图像内容。
5. 技术5: 自动驾驶技术的发展，利用传感器数据和机器学习算法进行实时路况分析和车辆控制，实现了无人驾驶的目标。
6. 技术6: 人机交互界面的设计和改进，如虚拟现实（VR）和增强现实（AR）技术，为用户提供了更加沉浸式的体验。
7. 技术7: 区块链技术在人工智能中的应用，可以提供去中心化的数据存储和交换平台，提高了数据安全性和隐私保护能力。
8. 技术8: 量子计算的发展，有望大幅提升计算能力和解决复杂问题的能力，对人工智能领域具有潜在影响。
9. 技术9: 联邦学习的方法，可以在不共享个人敏感信息的情况下进行模型训练和推理，提高了数据隐私保护的水平。
10. 技术10: 迁移学习的方法，可以通过将已


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)